# 시점 : step 1년
# 지역(17) : 서울, 부산, 대구, 인천, 광주, 대전, 울산, 세종, 경기, 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주
# 변수명(단위)

In [127]:
import pandas as pd
import numpy as np

In [128]:
# 지역명 변경 사전 : replace_region_names
region_mapping = {
    '서울특별시': '서울', '부산광역시': '부산', '대구광역시': '대구', '인천광역시': '인천',
    '광주광역시': '광주', '대전광역시': '대전', '울산광역시': '울산', '세종특별자치시': '세종',
    '경기도': '경기', '강원도': '강원', '강원특별자치도': '강원', '충청북도': '충북', '충청남도': '충남',
    '전라북도': '전북', '전북특별자치도': '전북', '전라남도': '전남', '경상북도': '경북', '경상남도': '경남',
    '제주도': '제주', '제주특별자치도': '제주'
}

# 지역명 변경 함수
def replace_region_names(df, region_mapping):
    df['지역'] = df['지역'].replace(region_mapping)
    return df

In [129]:
# 합계출산율
fertility_df = pd.read_csv('../origin/합계출산율_전국_년.csv', index_col=0)
fertility_df.rename(columns={'시군구별': '지역', '데이터': '합계출산율(명)'}, inplace=True)
fertility_df = fertility_df[fertility_df['지역'] != '전국']
fertility_df = replace_region_names(fertility_df, region_mapping)

fertility_df.reset_index(inplace=True)
fertility_df.to_csv('합계출산율.csv', index=False)

fertility_df['지역'].unique(), fertility_df['지역'].nunique(), fertility_df.head()

(array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
        '충남', '전북', '전남', '경북', '경남', '제주'], dtype=object),
 17,
      시점  지역 합계출산율(명)
 0  2000  서울    1.275
 1  2000  부산    1.235
 2  2000  대구    1.378
 3  2000  인천    1.473
 4  2000  광주    1.636)

In [130]:
# 아파트매매가격지수
apart_df = pd.read_csv('../origin/아파트매매가격지수_전국_월.csv', index_col=0)
apart_df.rename(columns={'행정구역별': '지역', '데이터': '아파트매매가격지수(%)'}, inplace=True)
apart_df = replace_region_names(apart_df, region_mapping)

apart_df.index = pd.to_datetime(apart_df.index.astype(str), format='%Y.%m', errors='coerce')
apart_df['연도'] = apart_df.index.year
yearly_df = apart_df.groupby(['연도', '지역'])['아파트매매가격지수(%)'].mean().reset_index()
yearly_df['아파트매매가격지수(%)'] = yearly_df['아파트매매가격지수(%)'].apply(lambda x: f"{x:.3f}")
yearly_df.rename(columns={'연도': '시점'}, inplace=True)

yearly_df.to_csv('아파트매매가격지수.csv', index=False)

yearly_df['지역'].unique(), yearly_df['지역'].nunique(), yearly_df.head()

(array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '울산', '인천',
        '전남', '전북', '제주', '충남', '충북', '세종'], dtype=object),
 17,
      시점  지역 아파트매매가격지수(%)
 0  2003  강원       67.350
 1  2003  경기       52.400
 2  2003  경남       64.850
 3  2003  경북       67.650
 4  2003  광주       53.450)

In [131]:
# 인구밀도 1차 : 선형보간
population_df = pd.read_csv('../origin/인구밀도_전국_년(5step).csv', index_col=0)
population_df.rename(columns={'행정구역별': '지역', '데이터': '인구밀도(명/km2)'}, inplace=True)
population_df = replace_region_names(population_df, region_mapping)

population_df_sorted = population_df.sort_values(by=['지역', '시점'])
population_df_sorted.reset_index(inplace=True)

def interpolate_group(group):
    group.set_index('시점', inplace=True)
    group = group.reindex(np.arange(2000, 2016, 1))
    group['지역'] = group['지역'].ffill().bfill()
    group['인구밀도(명/km2)'] = group['인구밀도(명/km2)'].interpolate(method='linear')
    group.reset_index(inplace=True)
    return group

# 새로운 데이터프레임 생성
interpolated_dfs = []

# 각 지역별로 데이터프레임을 분리하고 보간
for region in population_df_sorted['지역'].unique():
    region_df = population_df_sorted[population_df_sorted['지역'] == region]
    interpolated_df = interpolate_group(region_df)
    interpolated_dfs.append(interpolated_df)

# 보간된 데이터프레임들을 결합
final_interpolated_df = pd.concat(interpolated_dfs, ignore_index=True)

# 시점별로 데이터 정렬
final_interpolated_df = final_interpolated_df.sort_values(by=['시점', '지역']).reset_index(drop=True)

final_interpolated_df.head()



,시점,지역,인구밀도(명/km2)
0,2000,강원,89.7
1,2000,경기,886.4
2,2000,경남,283.2
3,2000,경북,143.2
4,2000,광주,2697.8


In [132]:
# 인구밀도 2차 : 실제 2016~2023 data와 보간 data 병합
population_df = population_df.reset_index()
population_16_23_df = population_df[population_df['시점'].between(2016, 2023)]

# 두 데이터프레임을 '시점'과 '지역' 기준으로 병합
merged_df = pd.merge(final_interpolated_df, population_16_23_df, on=['시점', '지역'], how='outer', suffixes=('_final', '_population'))
merged_df['인구밀도(명/km2)'] = merged_df['인구밀도(명/km2)_final'].combine_first(merged_df['인구밀도(명/km2)_population'])
merged_df['인구밀도(명/km2)'] = merged_df['인구밀도(명/km2)'].apply(lambda x: f"{x:.2f}")
merged_df = merged_df.drop(columns=['인구밀도(명/km2)_final', '인구밀도(명/km2)_population'])

merged_df.to_csv('인구밀도.csv', index=False)

merged_df.head(), merged_df.tail()


(     시점  지역 인구밀도(명/km2)
 0  2000  강원       89.70
 1  2000  경기      886.40
 2  2000  경남      283.20
 3  2000  경북      143.20
 4  2000  광주     2697.80,
        시점  지역 인구밀도(명/km2)
 403  2023  전남      143.70
 404  2023  전북      219.10
 405  2023  제주      365.80
 406  2023  충남      268.70
 407  2023  충북      221.60)

In [144]:
# 사교육참여율
edu_df = pd.read_csv('../origin/사교육참여율_전국_년.csv', index_col=0)
edu_df.reset_index(inplace=True)
edu_df.rename(columns={'지역별': '지역', '사교육참여율': '사교육참여율(%)'}, inplace=True)
edu_df = edu_df[['시점', '지역', '사교육참여율(%)']]

edu_df.to_csv('사교육참여율.csv', index=False)

edu_df['지역'].unique(), edu_df['지역'].nunique(), edu_df.head()

(array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
        '충남', '전북', '전남', '경북', '경남', '제주'], dtype=object),
 17,
      시점  지역  사교육참여율(%)
 0  2009  서울       79.6
 1  2010  서울       77.5
 2  2011  서울       77.0
 3  2012  서울       73.5
 4  2013  서울       75.0)

In [151]:
# 고용률
employment_df = pd.read_csv('../origin/청년고용률_전국_년.csv', index_col=0)
employment_df.rename(columns={'시도별': '지역', '데이터': '고용률(%)'}, inplace=True)
employment_df = replace_region_names(employment_df, region_mapping)
employment_df.drop(columns=['연령계층별'], inplace=True)
employment_df.reset_index(inplace=True)

employment_df.to_csv('고용률.csv', index=False)

employment_df['지역'].unique(), employment_df['지역'].nunique(), employment_df.head()

(array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남',
        '전북', '전남', '경북', '경남', '제주', '세종'], dtype=object),
 17,
      시점  지역  고용률(%)
 0  2000  서울    47.2
 1  2000  부산    39.6
 2  2000  대구    40.2
 3  2000  인천    44.9
 4  2000  광주    37.9)

In [156]:
# 여성초혼연령
age_df = pd.read_csv('../origin/평균초혼연령_전국_년.csv', index_col=0)
age_df.reset_index(inplace=True)
age_df.rename(columns={'시도별': '지역', '남편': '남성초혼연령(년)', '아내': '여성초혼연령(년)'}, inplace=True)
age_df = replace_region_names(age_df, region_mapping)
age_df.drop(columns=['남성초혼연령(년)'], inplace=True)

age_df.to_csv('여성초혼연령.csv', index=False)

age_df['지역'].unique(), age_df['지역'].nunique(), age_df.head()

(array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남',
        '전북', '전남', '경북', '경남', '제주', '세종'], dtype=object),
 17,
      시점  지역  여성초혼연령(년)
 0  2000  서울      27.25
 1  2000  부산      26.74
 2  2000  대구      26.51
 3  2000  인천      26.39
 4  2000  광주      26.17)

In [171]:
# 인터넷이용률
internet_df = pd.read_csv('../origin/인터넷이용률_전국_년.csv', index_col=0)
internet_df.reset_index(inplace=True)
internet_df.rename(columns={'특성별(2)': '지역', '이용': '인터넷이용률(%)'}, inplace=True)

conditions = ['충청', '전라', '경상']
internet_df.loc[internet_df['지역'].isin(conditions), '지역'] = internet_df['특성별(3)']

internet_df.drop(columns=['특성별(1)', '특성별(3)', '이용안함'], inplace=True)

internet_df.to_csv('인터넷이용률.csv', index=False)

internet_df['지역'].unique(), internet_df['지역'].nunique(), internet_df.head()

(array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남',
        '전북', '전남', '경북', '경남', '제주', '세종'], dtype=object),
 17,
      시점  지역  인터넷이용률(%)
 0  2008  서울       80.1
 1  2008  부산       74.2
 2  2008  대구       74.1
 3  2008  인천       80.7
 4  2008  광주       79.6)

In [179]:
# 근로시간
time_df = pd.read_csv('../origin/근로시간_전국_2011_2023.csv', index_col=0)
time_df.reset_index(inplace=True)
time_df.rename(columns={'지역별': '지역', '근로시간': '근로시간(시간)'}, inplace=True)
time_df['근로시간(시간)'] = time_df['근로시간(시간)'].apply(lambda x: f"{x:.1f}")

time_df.to_csv('근로시간.csv', index=False)

time_df.head()

,시점,지역,근로시간(시간)
0,2011,강원,2092.8
1,2011,경기,2086.8
2,2011,경남,2061.6
3,2011,경북,2029.2
4,2011,광주,2090.4
